# Lab 3

### Team Members:
 - Alex Lopez
 - Chris Haub
 - Erin McClure-Price
 - Chad Kwong

#### Library Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# import turicreate as tc
from sklearn.preprocessing import StandardScaler

#### Data Import

In [2]:
%%time

#Loading all data file separately 
vle = pd.read_csv('./anonymiseddata/vle.csv')
assessments = pd.read_csv('./anonymiseddata/assessments.csv')
courses = pd.read_csv('./anonymiseddata/courses.csv')
studentAssessments = pd.read_csv('./anonymiseddata/studentAssessment.csv')
studentInfo = pd.read_csv('./anonymiseddata/studentInfo.csv')
studentRegistration = pd.read_csv('./anonymiseddata/studentRegistration.csv')
studentVle = pd.read_csv('./anonymiseddata/studentVle.csv')

Wall time: 4.04 s


# Business Understanding 1 #
*Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?*

# Data Understanding 1 #
*Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?*

### Data Description

 The data comes in through a relational database schema with separated tables. We needed to merge these tables together into a singular dataframe
 
 - The data set appears to be for the UK
 - VLE stands for Virtual Learning Environment
 - VLE table records interactions with VLE 
 - 32,593 students
 - 22 courses
  
  
  we found that using all of the 32000 students was too intensive, so we sampled 4000 random students to create the dataframe from

In [3]:
%%time
# Randomly sampling 4,000 records StudentVle Table with Million Rows.
studentSample = studentInfo.sample(n=4000, random_state = 777)
print(studentSample.shape)

# vle
# assessments X
# courses X
# studentAssessments X
# studentInfo X
# studentRegistration X
# studentVleSample X

df = pd.merge(studentSample, studentVle, on=['code_module', 'code_presentation', 'id_student'])
df2 = pd.merge(studentAssessments, assessments, on='id_assessment')
df = pd.merge(df, df2, on=['code_module', 'code_presentation', 'id_student'], how='left')
df = pd.merge(df, courses, on= ['code_module', 'code_presentation'], how='left')
df = pd.merge(df, studentRegistration, on=['code_module', 'code_presentation', 'id_student'], how='left')
df = pd.merge(df, vle, on=['id_site', 'code_module', 'code_presentation'], how='left')
print(df.shape)
df.head()

(4000, 12)
(10744419, 28)
Wall time: 21.8 s


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,score,assessment_type,date_y,weight,module_presentation_length,date_registration,date_unregistration,activity_type,week_from,week_to
0,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,78.0,CMA,18.0,2.0,269,-70.0,32.0,homepage,NaN,NaN
1,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,78.0,CMA,18.0,2.0,269,-70.0,32.0,resource,NaN,NaN
2,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,78.0,CMA,18.0,2.0,269,-70.0,32.0,forumng,NaN,NaN
3,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,78.0,CMA,18.0,2.0,269,-70.0,32.0,forumng,NaN,NaN
4,CCC,2014J,134190,F,Scotland,A Level or Equivalent,30-40%,0-35,0,120,...,78.0,CMA,18.0,2.0,269,-70.0,32.0,homepage,NaN,NaN


### Missing Values

In [4]:
print('Missing values: \n', df.isnull().sum() * 100 / len(df),'\n\n')
df = df.drop(columns = ['week_from', 'week_to', 'date_unregistration'])
print('Missing values after dopping columns: \n', df.isnull().sum() * 100 / len(df))

Missing values: 
 code_module                    0.000000
code_presentation              0.000000
id_student                     0.000000
gender                         0.000000
region                         0.000000
highest_education              0.000000
imd_band                       6.064051
age_band                       0.000000
num_of_prev_attempts           0.000000
studied_credits                0.000000
disability                     0.000000
final_result                   0.000000
id_site                        0.000000
date_x                         0.000000
sum_click                      0.000000
id_assessment                  0.117233
date_submitted                 0.117233
is_banked                      0.117233
score                          0.160958
assessment_type                0.117233
date_y                         1.879394
weight                         0.117233
module_presentation_length     0.000000
date_registration              0.001889
date_unregistration   

From the cell above, we went ahead and dropped the week_from, week_to, and date_unregistration columns as these all were mostly missing values. These columns that were dropped could be used to analyze the population of the data that are not registered.

Below we can analyze the missing values for IMD Band and potentially fill those with values. Otherwise we need to justify dropping these rows.

In [5]:
missingIMDBand = df[df['imd_band'].isnull()]
print(missingIMDBand.region.value_counts() * 100 / len(missingIMDBand))

print(df.region.value_counts())

North Region            69.656525
Ireland                 24.472218
West Midlands Region     3.779773
South Region             2.091484
Name: region, dtype: float64
Scotland                1181415
London Region           1177689
South Region            1152974
East Anglian Region     1041307
North Western Region     803293
West Midlands Region     796770
South East Region        766654
South West Region        752397
East Midlands Region     742327
North Region             694503
Wales                    598712
Yorkshire Region         581790
Ireland                  454588
Name: region, dtype: int64


### Outliers

### One Hot Encoding

In [6]:
df.dtypes

code_module                    object
code_presentation              object
id_student                      int64
gender                         object
region                         object
highest_education              object
imd_band                       object
age_band                       object
num_of_prev_attempts            int64
studied_credits                 int64
disability                     object
final_result                   object
id_site                         int64
date_x                          int64
sum_click                       int64
id_assessment                 float64
date_submitted                float64
is_banked                     float64
score                         float64
assessment_type                object
date_y                        float64
weight                        float64
module_presentation_length      int64
date_registration             float64
activity_type                  object
dtype: object

In [7]:
# View unique values in each column
# for col in df:
#   print(df[col].unique())

In [8]:
#Creating variables for indexing continuous and categorical variables
conCol = ['id_student', 'id_site', 'date_x', 'sum_click', 'num_of_prev_attempts', 
          'studied_credits', 'id_assessment ', 'date_submitted', 'is_banked', 'score', 'date_y', 'weight']

catCol = ['code_module', 'code_presentation', 'gender', 'region', 'highest_education', 
          'imd_band', 'age_band', 'disability', 'final_result', 'assessment_type' ]

In [9]:
%%time
#Declaring Scalar object
scl_obj = StandardScaler()

#One hot encoding of ALL categorical variables
OneHotDF = pd.get_dummies(df[catCol],drop_first=False)

#Scaling non-encoded data
conVar = df.select_dtypes(exclude='object')
colnames = pd.Series(conVar.columns)
conVarScaled = scl_obj.fit_transform(conVar)
conVarScaled = pd.DataFrame(data = conVarScaled, columns= colnames)

#Combining with continuous variables from cleaned dataset
OneHotDF = OneHotDF.reset_index()
OneHotDF.pop('index')
OneHotDF = pd.concat([conVarScaled,OneHotDF], axis = 1)
OneHotDF.head()

#https://github.com/jakemdrew/DataMiningNotebooks/blob/master/01.%20Pandas.ipynb


Wall time: 32.9 s


,id_student,num_of_prev_attempts,studied_credits,id_site,date_x,sum_click,id_assessment,date_submitted,is_banked,score,...,age_band_55<=,disability_N,disability_Y,final_result_Distinction,final_result_Fail,final_result_Pass,final_result_Withdrawn,assessment_type_CMA,assessment_type_Exam,assessment_type_TMA
0,-1.052995,-0.286529,1.167467,1.329246,-1.530831,-0.281463,-0.493133,-1.400989,-0.053717,-0.069702,...,0,1,0,0,0,0,1,1,0,0
1,-1.052995,-0.286529,1.167467,1.331293,-1.530831,-0.281463,-0.493133,-1.400989,-0.053717,-0.069702,...,0,1,0,0,0,0,1,1,0,0
2,-1.052995,-0.286529,1.167467,1.329284,-1.294020,-0.281463,-0.493133,-1.400989,-0.053717,-0.069702,...,0,1,0,0,0,0,1,1,0,0
3,-1.052995,-0.286529,1.167467,1.329284,-1.294020,-0.181529,-0.493133,-1.400989,-0.053717,-0.069702,...,0,1,0,0,0,0,1,1,0,0
4,-1.052995,-0.286529,1.167467,1.329246,-1.294020,0.218209,-0.493133,-1.400989,-0.053717,-0.069702,...,0,1,0,0,0,0,1,1,0,0


# Data Understanding 2 #
*Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.*

# Modeling and Evaluation 1 #
*Train and adjust parameters*

# Modeling and Evaluation 2	#
*Evaluate and Compare*

# Modeling and Evaluation 3 #
*Visualize Results*

# Modeling and Evaluation 4 #
*Summarize the Ramifications*

# Deployment #
*Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?*

# Exceptional Work #